# Chapter 3: NVIDIA Isaac Platform

This notebook explores the NVIDIA Isaac platform for robotics, including Isaac ROS packages and Isaac Sim. We'll learn how to leverage NVIDIA's GPU computing capabilities for accelerated robotics applications.

In [ ]:
# Configuration cell - select execution mode
EXECUTION_MODE = "simulation"  # Options: "hardware", "simulation"

print(f"Isaac Chapter 3: Running in {EXECUTION_MODE} mode")
print("Initializing Isaac environment...")

# Import required libraries
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from PIL import Image

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")
if cuda_available:
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

print("Isaac environment setup complete!")

## 1. Understanding the NVIDIA Isaac Platform

NVIDIA Isaac is a comprehensive robotics platform that includes hardware, software, and simulation tools. In this notebook, we'll explore Isaac ROS packages and how they accelerate robotics applications.

In [ ]:
# Create mock Isaac ROS packages for simulation mode
if EXECUTION_MODE == "simulation":
    print("Using Isaac simulation mode - no physical hardware required")
    
    # Mock Isaac ROS classes
    class MockIsaacROSNode:
        def __init__(self, name):
            self.name = name
            print(f"Mock Isaac ROS Node '{name}' initialized")
        
        def create_publisher(self, msg_type, topic_name, qos_profile=None):
            print(f"Mock Isaac publisher created for topic: {topic_name}")
            return MockIsaacPublisher(topic_name)
        
        def create_subscription(self, msg_type, topic_name, callback, qos_profile=None):
            print(f"Mock Isaac subscription created for topic: {topic_name}")
            return MockIsaacSubscription(topic_name, callback)
    
    class MockIsaacPublisher:
        def __init__(self, topic_name):
            self.topic_name = topic_name
        
        def publish(self, msg):
            print(f"Isaac publishing to {self.topic_name}: {msg}")
    
    class MockIsaacSubscription:
        def __init__(self, topic_name, callback):
            self.topic_name = topic_name
            self.callback = callback
    
    # Isaac-specific mock functions
    class IsaacROSPython:
        @staticmethod
        def init():
            print("Mock Isaac ROS initialized")
        
        @staticmethod
        def spin_once(node, timeout_sec=None):
            print("Mock Isaac spin_once called")
            time.sleep(0.1)
        
        @staticmethod
        def shutdown():
            print("Mock Isaac ROS shutdown")
    
    # Assign mock classes
    IsaacNode = MockIsaacROSNode
    IsaacROS = IsaacROSPython()
    
    # Mock Isaac perception packages
    class MockImagePipeline:
        def __init__(self):
            print("Mock Isaac Image Pipeline initialized")
        
        def process(self, image):
            # Simulate GPU-accelerated image processing
            print(f"Processing image with mock Isaac pipeline: {image.shape if hasattr(image, 'shape') else 'image'}")
            return image  # Return processed image
    
    class MockDetectionPipeline:
        def __init__(self):
            print("Mock Isaac Detection Pipeline initialized")
        
        def detect(self, image):
            # Simulate object detection
            print(f"Detecting objects in image with mock Isaac pipeline")
            # Return mock detections
            return [{'class': 'object', 'confidence': 0.9, 'bbox': [0.1, 0.1, 0.5, 0.5]}]
    
    # Create mock pipelines
    image_pipeline = MockImagePipeline()
    detection_pipeline = MockDetectionPipeline()
    
    print("Mock Isaac components initialized")
else:
    print("Using Isaac hardware mode - connecting to Isaac platform")
    # In a real environment, we would import actual Isaac packages
    # import isaac_ros_python
    # from isaac_ros.perception import ImagePipeline, DetectionPipeline
    
    # For this simulation, we'll use the mock components
    image_pipeline = MockImagePipeline()
    detection_pipeline = MockDetectionPipeline()

## 2. Isaac ROS Packages for Perception

Isaac ROS provides hardware-accelerated perception packages. Let's explore how these work with simulated data.

In [ ]:
# Create a mock Isaac perception node
class IsaacPerceptionNode(IsaacNode):
    def __init__(self):
        super().__init__('isaac_perception')
        
        # Create publishers and subscribers
        self.image_pub = self.create_publisher(type('Image', (), {}), 'camera/image_raw')
        self.detection_pub = self.create_publisher(type('Detection2DArray', (), {}), 'detections')
        
        # Initialize Isaac perception components
        self.image_pipeline = MockImagePipeline()
        self.detection_pipeline = MockDetectionPipeline()
        
        print("Isaac Perception Node initialized with accelerated pipelines")
    
    def process_image(self, image_data):
        # Process image using Isaac pipeline
        print(f"Processing image data with Isaac pipeline")
        processed_image = self.image_pipeline.process(image_data)
        
        # Run object detection
        detections = self.detection_pipeline.detect(processed_image)
        
        # Publish results
        self.image_pub.publish(processed_image)
        self.detection_pub.publish(detections)
        
        return detections

# Create the Isaac perception node
perception_node = IsaacPerceptionNode()
print("Isaac Perception Node created successfully")

## 3. Creating Simulated Camera Data

Let's create some simulated camera data to process with our Isaac perception pipeline.

In [ ]:
# Generate a simulated camera image
def create_simulated_camera_image(width=640, height=480, channels=3):
    """Create a simulated camera image with geometric shapes"""
    image = np.zeros((height, width, channels), dtype=np.uint8)
    
    # Add some geometric shapes to represent objects
    # Rectangle
    image[100:200, 150:250, :] = [255, 0, 0]  # Red rectangle
    
    # Circle
    center_x, center_y = 400, 150
    radius = 50
    y, x = np.ogrid[:height, :width]
    mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
    image[mask] = [0, 255, 0]  # Green circle
    
    # Triangle
    triangle_points = np.array([[300, 300], [350, 350], [250, 350]])
    # Fill the triangle
    for i in range(250, 351):
        for j in range(300, 351):
            # Simple triangle fill algorithm
            if i >= 250 and i <= 350 and j >= 300 and j <= 350:
                if j <= i + 50 and j >= -i + 550:
                    image[j, i] = [0, 0, 255]  # Blue triangle
    
    return image

# Create simulated camera data
simulated_image = create_simulated_camera_image()
print(f"Simulated camera image created with shape: {simulated_image.shape}")

# Visualize the simulated image
plt.figure(figsize=(10, 8))
plt.imshow(simulated_image)
plt.title('Simulated Camera Image for Isaac Processing')
plt.axis('off')
plt.show()

## 4. Processing with Isaac Perception Pipeline

Now let's process our simulated image using the Isaac perception pipeline.

In [ ]:
# Process the simulated image with Isaac pipeline
print("Processing simulated image with Isaac perception pipeline...")

# In simulation mode, we'll mock the processing
if EXECUTION_MODE == "simulation":
    # Simulate GPU-accelerated processing
    print("Simulating GPU-accelerated image processing...")
    time.sleep(0.5)  # Simulate processing time
    
    # Mock processing results
    detections = [
        {'class': 'rectangle', 'confidence': 0.95, 'bbox': [150, 100, 250, 200]},
        {'class': 'circle', 'confidence': 0.92, 'bbox': [350, 100, 450, 200]},
        {'class': 'triangle', 'confidence': 0.88, 'bbox': [250, 300, 350, 350]}
    ]
    
    print(f"Mock detections: {detections}")
else:
    # In hardware mode, we would use the actual Isaac pipeline
    detections = perception_node.process_image(simulated_image)
    print(f"Real detections: {detections}")

# Visualize the detections on the image
fig, ax = plt.subplots(figsize=(12, 8))
ax.imshow(simulated_image)
ax.set_title('Isaac Perception Results - Detected Objects')

# Draw bounding boxes for detections
for detection in detections:
    bbox = detection['bbox']
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    
    # Draw rectangle
    rect = plt.Rectangle((x1, y1), width, height, 
                        linewidth=2, edgecolor='yellow', facecolor='none')
    ax.add_patch(rect)
    
    # Add label
    ax.text(x1, y1 - 10, f"{detection['class']} ({detection['confidence']:.2f})", 
           bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7),
           fontsize=10, fontweight='bold')

plt.axis('off')
plt.tight_layout()
plt.show()

print(f"Processed image with {len(detections)} objects detected")

## 5. Isaac Navigation Stack

Let's explore the Isaac navigation stack with a simulated navigation scenario.

In [ ]:
# Create a mock Isaac navigation node
class IsaacNavigationNode(IsaacNode):
    def __init__(self):
        super().__init__('isaac_navigation')
        
        # Initialize Isaac navigation components
        self.path_planner = self._init_path_planner()
        self.controller = self._init_controller()
        
        print("Isaac Navigation Node initialized with accelerated components")
    
    def _init_path_planner(self):
        print("Initializing Isaac Path Planner (GPU-accelerated)")
        return MockPathPlanner()
    
    def _init_controller(self):
        print("Initializing Isaac Controller (GPU-accelerated)")
        return MockController()
    
    def navigate_to_pose(self, goal_x, goal_y, goal_theta=0):
        print(f"Planning path to goal: ({goal_x}, {goal_y}, {goal_theta})")
        
        # Plan path using Isaac's accelerated planner
        path = self.path_planner.plan_path(goal_x, goal_y)
        
        # Execute navigation using Isaac's controller
        success = self.controller.follow_path(path)
        
        return success

# Mock navigation components
class MockPathPlanner:
    def __init__(self):
        self.name = "Mock Isaac Path Planner"
        print(f"{self.name} initialized")
    
    def plan_path(self, goal_x, goal_y):
        print(f"Planning path to ({goal_x}, {goal_y}) using Isaac acceleration")
        # Simulate GPU-accelerated path planning
        time.sleep(0.2)  # Simulate planning time
        
        # Create a simple path
        start_x, start_y = 0, 0
        path = []
        steps = 20
        for i in range(steps + 1):
            t = i / steps
            x = start_x + (goal_x - start_x) * t
            y = start_y + (goal_y - start_y) * t
            path.append((x, y))
        
        print(f"Path planned with {len(path)} waypoints")
        return path

class MockController:
    def __init__(self):
        self.name = "Mock Isaac Controller"
        print(f"{self.name} initialized")
    
    def follow_path(self, path):
        print(f"Following path with {len(path)} waypoints using Isaac control")
        # Simulate GPU-accelerated control
        for i, waypoint in enumerate(path):
            print(f"Moving to waypoint {i+1}/{len(path)}: ({waypoint[0]:.2f}, {waypoint[1]:.2f})", end='\r')
            time.sleep(0.05)  # Simulate movement time
        print(f"\nNavigation completed successfully")
        return True

# Create the Isaac navigation node
navigation_node = IsaacNavigationNode()
print("Isaac Navigation Node created successfully")

## 6. GPU-Accelerated Navigation Visualization

Let's visualize the navigation process with simulated robot movement.

In [ ]:
# Simulate a navigation scenario
print("Starting Isaac navigation simulation...")

# Define navigation goals
goals = [
    (5.0, 5.0),    # Goal 1
    (8.0, 2.0),    # Goal 2
    (3.0, 8.0),    # Goal 3
    (10.0, 10.0)   # Goal 4
]

# Create visualization
fig, ax = plt.subplots(figsize=(12, 10))
ax.set_xlim(-1, 12)
ax.set_ylim(-1, 12)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_title('Isaac Navigation Simulation - GPU-Accelerated Path Planning')

# Plot goals
goal_x_coords = [goal[0] for goal in goals]
goal_y_coords = [goal[1] for goal in goals]
ax.plot(goal_x_coords, goal_y_coords, 'ro', markersize=10, label='Goals', markeredgecolor='black')

# Annotate goals
for i, goal in enumerate(goals):
    ax.text(goal[0] + 0.2, goal[1] + 0.2, f'Goal {i+1}', fontsize=10, fontweight='bold')

# Start position
start_x, start_y = 0, 0
ax.plot(start_x, start_y, 'go', markersize=12, label='Start', markeredgecolor='black')

# Simulate navigating to each goal
all_paths = []
current_pos = (start_x, start_y)

for i, goal in enumerate(goals):
    print(f"\nNavigating to Goal {i+1}: ({goal[0]}, {goal[1]})")
    
    # Plan and follow path
    path = navigation_node.path_planner.plan_path(goal[0], goal[1])
    navigation_node.controller.follow_path(path)
    
    # Store path for visualization
    all_paths.extend(path)
    
    # Plot the path
    path_x = [point[0] for point in path]
    path_y = [point[1] for point in path]
    ax.plot(path_x, path_y, 'b-', linewidth=2, alpha=0.7, label=f'Path to Goal {i+1}' if i == 0 else "")
    
    # Mark the achieved goal
    ax.plot(goal[0], goal[1], 'r*', markersize=16, markeredgecolor='black')

# Mark the final position
final_pos = goals[-1]
ax.plot(final_pos[0], final_pos[1], 'r*', markersize=20, markeredgecolor='black', label='Final Position')

ax.legend()
plt.tight_layout()
plt.show()

print(f"\nNavigation simulation completed! Visited {len(goals)} goals.")

## 7. Isaac Sim Integration

Let's explore how Isaac integrates with simulation environments for training and testing.

In [ ]:
# Create a mock Isaac Sim environment
class IsaacSimEnvironment:
    def __init__(self):
        self.name = "Mock Isaac Sim Environment"
        print(f"{self.name} initialized")
        
        # Simulate Isaac Sim components
        self.physics = MockPhysicsEngine()
        self.renderer = MockRenderer()
        self.ros_bridge = MockROSBridge()
        
        print("Isaac Sim components initialized")
    
    def create_robot(self, robot_type="differential_drive"):
        print(f"Creating {robot_type} robot in Isaac Sim")
        return MockRobot(robot_type)
    
    def simulate(self, steps=100):
        print(f"Running Isaac Sim for {steps} steps with GPU acceleration")
        for step in range(steps):
            if step % 20 == 0:
                print(f"Sim step {step}/{steps}", end='\r')
            self.physics.step()
            self.renderer.render()
            time.sleep(0.001)  # Simulate simulation time
        print(f"\nSimulation completed")
    
    def get_sensor_data(self, robot):
        # Simulate getting sensor data from the robot
        return {
            'camera': np.random.rand(480, 640, 3),  # Simulated camera data
            'lidar': np.random.rand(360),            # Simulated LiDAR data
            'imu': {'linear_accel': [0.1, 0.2, 9.8], 'angular_vel': [0.01, 0.02, 0.03]},
            'wheel_encoders': {'left': 1.2, 'right': 1.1}
        }

# Mock Isaac Sim components
class MockPhysicsEngine:
    def __init__(self):
        print("Mock Physics Engine initialized")
    
    def step(self):
        # Simulate physics step
        pass

class MockRenderer:
    def __init__(self):
        print("Mock Renderer initialized")
    
    def render(self):
        # Simulate rendering
        pass

class MockROSBridge:
    def __init__(self):
        print("Mock ROS Bridge initialized")
    
    def publish_sensor_data(self, data):
        print(f"Publishing sensor data via ROS Bridge: {list(data.keys())}")

class MockRobot:
    def __init__(self, robot_type):
        self.type = robot_type
        self.name = f"Mock {robot_type} Robot"
        print(f"{self.name} created with sensors and actuators")
    
    def move(self, linear_vel, angular_vel):
        print(f"Moving robot with linear_vel={linear_vel}, angular_vel={angular_vel}")

# Create Isaac Sim environment
isaac_sim = IsaacSimEnvironment()
print("Isaac Sim environment created successfully")

# Create a robot in the simulation
robot = isaac_sim.create_robot("differential_drive")

# Run a short simulation
print("\nStarting Isaac Sim simulation...")
isaac_sim.simulate(steps=50)

# Get sensor data from the simulated robot
sensor_data = isaac_sim.get_sensor_data(robot)
print(f"\nReceived sensor data from simulated robot:")
for sensor_type, data in sensor_data.items():
    if isinstance(data, dict):
        print(f"  {sensor_type}: {list(data.keys())}")
    else:
        print(f"  {sensor_type}: shape {getattr(data, 'shape', len(data) if hasattr(data, '__len__') else 'N/A')}")

## Summary

In this notebook, we've covered:
1. The NVIDIA Isaac platform and its components
2. Isaac ROS packages for accelerated perception
3. GPU-accelerated navigation stack
4. Isaac Sim integration for simulation and training
5. Processing of sensor data with Isaac pipelines
6. Visualization of Isaac-powered robotics applications

The NVIDIA Isaac platform provides powerful tools for robotics development, leveraging GPU acceleration for perception, navigation, and simulation. These tools enable more sophisticated robotics applications than would be possible with CPU-only processing.

In [ ]:
# Clean up
print("\nChapter 3 complete! You've learned about the NVIDIA Isaac platform.")
print("Isaac concepts covered:")
print("- Isaac ROS packages for accelerated perception")
print("- GPU-accelerated navigation")
print("- Isaac Sim integration")
print("- Sensor processing with Isaac pipelines")

if cuda_available:
    print("\nCUDA acceleration available for Isaac applications!")
else:
    print("\nCUDA acceleration not available - Isaac simulation mode active")